## Taxi Trajectory Prediction

Here are the links to papers and github repo with solution

https://arxiv.org/pdf/1508.00021.pdf  
https://github.com/adbrebs/taxi

In [1]:
! ls /home/ubuntu/taxi

arrival-clusters.pkl			    taxi1.ipynb   train.csv
arrivals.pkl				    test2pickle   val_idx.pkl
__MACOSX				    test.csv	  X_train.pkl
metaData_taxistandsID_name_GPSlocation.csv  tmp		  X_valid.pkl
models					    train2pickle


In [2]:
import torch

In [3]:
import pickle
from fastai.imports import *
from fastai.structured import *
from fastai.column_data import *
import torch.nn as nn
from fastai.dataset import *
from fastai.learner import *

In [4]:
path = "/home/ubuntu/taxi/"

In [47]:
# import zipfile
# zip_ref = zipfile.ZipFile('train.csv.zip', 'r')
# zip_ref.extractall(path)
# zip_ref.close()

In [48]:
train = pd.read_csv("train.csv")

In [49]:
test = pd.read_csv("test.csv")

In [50]:
train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [51]:
list(train.POLYLINE[:1])

['[[-8.618643,41.141412],[-8.618499,41.141376],[-8.620326,41.14251],[-8.622153,41.143815],[-8.623953,41.144373],[-8.62668,41.144778],[-8.627373,41.144697],[-8.630226,41.14521],[-8.632746,41.14692],[-8.631738,41.148225],[-8.629938,41.150385],[-8.62911,41.151213],[-8.629128,41.15124],[-8.628786,41.152203],[-8.628687,41.152374],[-8.628759,41.152518],[-8.630838,41.15268],[-8.632323,41.153022],[-8.631144,41.154489],[-8.630829,41.154507],[-8.630829,41.154516],[-8.630829,41.154498],[-8.630838,41.154489]]']

In [52]:
list(test.POLYLINE[:1])

['[[-8.585676,41.148522],[-8.585712,41.148639],[-8.585685,41.148855],[-8.58573,41.148927],[-8.585982,41.148963],[-8.586396,41.148954],[-8.586072,41.14872],[-8.586324,41.147847],[-8.586999,41.14746],[-8.586576,41.147154],[-8.584884,41.146623]]']

In [53]:
test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[..."
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-..."
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-..."
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[..."
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]"


In [54]:
meta = pd.read_csv("metaData_taxistandsID_name_GPSlocation.csv")

In [55]:
meta.head()

,ID,Descricao,Latitude,Longitude
0,1,Agra,41.1771457135,-8.609670
1,2,Alameda,41.15618964,-8.591064
2,3,Aldoar,41.1705249231,-8.665876
3,4,Alfândega,41.1437639911,-8.621803
4,5,Amial,41.1835097223,-8.612726


In [56]:
meta.shape, test.shape, train.shape

((63, 4), (320, 9), (1710670, 9))

Ok. So we have 1.7 million training rides and only 320 test rides. Metadata is just having name of cities for lat/long. 

In [57]:
train.dtypes

TRIP_ID           int64
CALL_TYPE        object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID           int64
TIMESTAMP         int64
DAY_TYPE         object
MISSING_DATA       bool
POLYLINE         object
dtype: object

### Step 1. Feature extraction

Winner's process to convert csv to hdf5 and extract features. I would prefer working on `pandas` for the same as it will allow me to look at how data looks. Replicating all steps present in `csv_to_hdf5.py` to make pandas df

In [58]:
train2 = train.copy()
test2 = test.copy()
meta2 = meta.copy()

Below code proves that all the taxi ids in test set are also present in training set. We don't have any new taxi id in test data. But there are **20 origin calls** that are not in training data but are in test data. 

In [6]:
s=0
for l in test.TAXI_ID.unique():
    if l not in train.TAXI_ID.unique():
        s = s+1
p=0
g = []
for l in test.ORIGIN_CALL.unique():
    if l not in train.ORIGIN_CALL.unique():
        p = p+1
        g.append(l)
s,p

`g` as saved above has list of origin_calls that are in test but not in train

In [62]:
g[1:]

[85698.0,
 85224.0,
 86098.0,
 86436.0,
 74478.0,
 81753.0,
 67507.0,
 65127.0,
 66996.0,
 72185.0,
 75460.0,
 73870.0,
 72911.0,
 73071.0,
 75037.0,
 80148.0,
 66812.0,
 70885.0,
 76232.0]

Let's see how many unique `origin_calls` and `taxi_ids` do we have in both training and test set

In [63]:
len(train.TAXI_ID.unique()), len(test.TAXI_ID.unique()), len(train.ORIGIN_CALL.unique()), len(test.ORIGIN_CALL.unique())

(448, 244, 57106, 61)

So basically, approximately half of `taxi_ids` from training are there in test data.

Pre-calculated below on train set


In [5]:
lat_mean = 41.15731
lat_std = 0.074120656
long_mean = -8.6161413
long_std = 0.057200309

Making a new feature with only first 5 and last 5 coordinates. In cases where we don't have enough coordinates, we do `edge` padding (i.e. just adding first and last values in leftmost and rightmost sides respectively)

In [70]:
def start_stop_inputs(k, train, test):
    
    """
    Function used to take first k and last k coordinates from given polyline.
    For train - if total coordinates < 2 then return none, if > 2*k then return first k and last k
    and if > 2 but < 2*k then do edge padding and return.
    
    """
    result_train = []
    result_test = []
    
    for l in train[['LONGITUDE','LATITUDE']].iterrows():
        if len(l[1][0]) < 2 or len(l[1][1]) < 2:
            result_train.append(np.nan)
        elif len(l[1][0][:-1]) >= 2*k:
            result_train.append(np.concatenate([l[1][0][0:k],l[1][0][-k:],
                                          l[1][1][0:k],l[1][1][-k:]]).flatten())
        else:
            l1 = np.lib.pad(l[1][0][:-1], (0,4*k-len(l[1][0][:-1])), mode='edge')
            l2 = np.lib.pad(l[1][1][:-1], (0,4*k-len(l[1][1][:-1])), mode='edge')
            result_train.append(np.concatenate([l1[0:k],l1[-k:],l2[0:k],l2[-k:]]).flatten())

    for l in test[['LONGITUDE','LATITUDE']].iterrows(): 
        if len(l[1][0]) < 1 or len(l[1][1]) < 1:
            result_test.append(np.nan)
        elif len(l[1][0]) >= 2*k:
            result_test.append(np.concatenate([l[1][0][0:k],l[1][0][-k:],l[1][1][0:k],l[1][1][-k:]]).flatten())
        else:
            l1 = np.lib.pad(l[1][0], (0,4*k-len(l[1][0])), mode='edge')
            l2 = np.lib.pad(l[1][1], (0,4*k-len(l[1][1])), mode='edge')
            result_test.append(np.concatenate([l1[0:k],l1[-k:],l2[0:k],l2[-k:]]).flatten())
    
    return pd.Series(result_train), pd.Series(result_test)

In [75]:
import ast
def feature_ext(train, test):   
    """
    Function used to extract model features.
    • extract lat/long from polyline
    • lat/long normalized using pre-calculated mean and std
    • origin_call and taxi_id converted to continuous numbers (0 to ...) to get embeddings later
    • categories convert to cat codes
    
    """
    
    # origin_call -- train
    train.ORIGIN_CALL.fillna(value = -1, inplace=True)
    id_uniq = train.ORIGIN_CALL.unique()
    
    origin_call_dict = {o:i for i,o in enumerate(id_uniq)}

    train['ORIGIN_CALL'] = train.ORIGIN_CALL.apply(lambda x: 0 if x == -1 
                                                   or x == '' else origin_call_dict[x])
    
    
    # origin_call --test (g list of ids in test not in train)
    test.ORIGIN_CALL.fillna(value = -1, inplace=True)

    test['ORIGIN_CALL'] = test.ORIGIN_CALL.apply(lambda x: 0 if x == -1 or x == '' 
                                              else (-2 if x in g[1:] else origin_call_dict[x]))
    
    
    # origin_stand
    train['ORIGIN_STAND']= pd.Series([0 if pd.isnull(x) or x=='' else int(x) for x in train["ORIGIN_STAND"]])
    test['ORIGIN_STAND']= pd.Series([0 if pd.isnull(x) or x=='' else int(x) for x in test["ORIGIN_STAND"]])
    
    # taxi_id
    id_uniq = train.TAXI_ID.unique()
    taxi_id_dict = {o:i for i,o in enumerate(id_uniq)}
    train['TAXI_ID'] = train.TAXI_ID.apply(lambda x: taxi_id_dict[x])
    test['TAXI_ID'] = test.TAXI_ID.apply(lambda x: taxi_id_dict[x])

    # day_type and call_type
    # want 0 for A, 1 for B and 2 for C
    train['DAY_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in train['DAY_TYPE']])
    train['CALL_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in train['CALL_TYPE']])
    test['DAY_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in test['DAY_TYPE']])
    test['CALL_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in test['CALL_TYPE']])
    
    
    # polyline
    polyline1 = pd.Series([ast.literal_eval(x) for x in train['POLYLINE']])
    polyline2 = pd.Series([ast.literal_eval(x) for x in test['POLYLINE']])
    
    # latitude and longitude
    train['LATITUDE'] = pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in polyline1])
    train['LONGITUDE'] = pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in polyline1])
    
    test['LATITUDE'] = pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in polyline2])
    test['LONGITUDE'] = pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in polyline2])
    
    # target variable i.e. last coordinates
    train['TARGET'] = pd.Series([[l[1][0][-1], l[1][1][-1]] if len(l[1][0]) > 1 
                                else np.nan for l in train[['LONGITUDE','LATITUDE']].iterrows()])

    # normalized lat long. We need normalized data for neural nets and this is like cont. variable (not cat)
    train['LATITUDE'] = pd.Series([(t-lat_mean)/lat_std for t in train['LATITUDE']])
    test['LATITUDE'] = pd.Series([(t-lat_mean)/lat_std for t in test['LATITUDE']])
    
    train['LONGITUDE'] = pd.Series([(t-long_mean)/long_std for t in train['LONGITUDE']])
    test['LONGITUDE'] = pd.Series([(t-long_mean)/long_std for t in test['LONGITUDE']])

    # first 5 and last 5 features
    train['COORD_FEATURES'], test['COORD_FEATURES'] = start_stop_inputs(5, train, test)
    
    # day of week variable
    train['DAY_OF_WEEK'] = pd.Series([datetime.datetime.fromtimestamp(t).weekday() for t in train['TIMESTAMP']])
    test['DAY_OF_WEEK'] = pd.Series([datetime.datetime.fromtimestamp(t).weekday() for t in test['TIMESTAMP']])

    # hour of day
    train['HOUR'] = pd.Series([datetime.datetime.fromtimestamp(t).hour for t in train['TIMESTAMP']])
    test['HOUR'] = pd.Series([datetime.datetime.fromtimestamp(t).hour for t in test['TIMESTAMP']])

    # week of year
    train['WEEK_OF_YEAR'] = pd.Series([datetime.datetime.fromtimestamp(t).isocalendar()[1] for t in train['TIMESTAMP']])
    test['WEEK_OF_YEAR'] = pd.Series([datetime.datetime.fromtimestamp(t).isocalendar()[1] for t in test['TIMESTAMP']])

        
    #data = data.dropna()

    return train, test

In [77]:
train2, test2 = train.copy(), test.copy()

In [ ]:
train2, test2 = feature_ext(train2, test2)

In [9]:
train2[:2]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,TARGET,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,1372636858620000589,2,0,0,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[-0.21451, -0.214974, -0.199688, -0.182087, -0...","[-0.0437321, -0.0412145, -0.0731591, -0.105104...","[-8.63084, 41.1545]","[-0.0437321, -0.0412145, -0.0731591, -0.105104...",0,0,27
1,1372637303620000596,1,0,7,1,1372637303,0,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[0.0339161, 0.0345337, 0.0378275, 0.0429227, 0...","[-0.414429, -0.423249, -0.455494, -0.494991, -...","[-8.66574, 41.1707]","[-0.414429, -0.423249, -0.455494, -0.494991, -...",0,0,27


In [8]:
test2[:2]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,T1,1,0,15,150,1408039037,0,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[...","[-0.118578, -0.116982, -0.1141, -0.113122, -0....","[0.532604, 0.531971, 0.532454, 0.531671, 0.527...","[0.532604, 0.531971, 0.532454, 0.531671, 0.527...",3,17,33
1,T2,1,0,57,306,1408038611,0,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-...","[-0.158413, -0.158258, -0.155736, -0.150024, -...","[0.0920491, 0.0923659, 0.0915823, 0.0996017, 0...","[0.0920491, 0.0923659, 0.0915823, 0.0996017, 0...",3,17,33


Saving transformed files as `pickle`

Saving files

In [101]:
train2.to_pickle(path+'train2pickle')
test2.to_pickle(path+'test2pickle')

Loading train and test. **Can directly run this**

In [6]:
train2 = pd.read_pickle('train2pickle')
test2 = pd.read_pickle('test2pickle')

After running `fit`, I got to know that we can not make predictions for **embeddings = -2** as it is not being trained. So there are only **20** such records, I am changing those as **0** in test set

In [7]:
test2.ORIGIN_CALL = test2.ORIGIN_CALL.apply(lambda x: 0 if x == -2 else x)

### Step 2: Meanshift clustering

** From the paper**

As the destination we aim to predict is composed of two scalar values (latitude and longitude), it is natural to have two output neurons. However, we found that it was difficult to train such a simple model because it does not take into account any prior information on the distribution of the data. To tackle this issue, we integrate prior knowledge of the destinations directly in the architecture of our model: instead of predicting directly the destination position, we use a predefined set (ci)1≤i≤C of a few thousand destination cluster centers and a hidden layer
that associates a scalar value (pi)i (similar to a probability) to each of these clusters. As the network must output a single destination position, for our output prediction yˆ, we compute a weighted average of the predefined destination cluster centers:

In [8]:
import os
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs
from itertools import cycle
import scipy.misc

In [8]:
# checking how to index and subset lat and long (lat is in 1 and long in 0th)
train2.TARGET[:10][1][1], train2.TARGET[:10][1][0]

(41.17067, -8.66574)

We don't have polyline data for all the training data. For those, we have `nan` in `TARGET` variable and those won't contribute in centroid calculation. 

In [40]:
print("Generating arrival point list")
dests = []

# for l in train2[['LONGITUDE','LATITUDE']].iterrows():
#     if len(l[1][0]) == 0 or len(l[1][1]) == 0: continue
#     dests.append((l[1][1][-1],  l[1][0][-1])) # latitude, longitude

target = train2.TARGET.fillna(0)    
for l in target:
    if l == 0 : continue
    dests.append((l[1],  l[0])) # latitude, longitude
    
pts = np.array(dests)

with open(os.path.join(path, "arrivals.pkl"), "wb") as f:
    pickle.dump(pts, f, protocol=pickle.HIGHEST_PROTOCOL)

print("Doing clustering")
bw = estimate_bandwidth(pts, quantile=.1, n_samples=1000)
print(bw)
bw = 0.001 

ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(pts)
cluster_centers = ms.cluster_centers_

print("Clusters shape: ", cluster_centers.shape)

with open(os.path.join(path, "arrival-clusters.pkl"), "wb") as f:
    pickle.dump(cluster_centers, f, protocol=pickle.HIGHEST_PROTOCOL)

Generating arrival point list
Doing clustering
0.0215466177411
Clusters shape:  (3422, 2)


I tried above code for --  
1. scaled coordinates. It doesn't give even close number of clusters. got **22644 clusters**. Not quite right. 
2. unscaled coordinates. Got **3422 clusters**. Winner had 3392 clusters. CLOSE and will keep it

### Step 3: Making Validation Split

As given in winner's solution, we are taking validation set as last year's data with same timestamp as of test data timestamps

In [9]:
# Cuts of the test set minus 1 year
cuts = [
    1376503200, # 2013-08-14 18:00
    1380616200, # 2013-10-01 08:30
    1381167900, # 2013-10-07 17:45
    1383364800, # 2013-11-02 04:00
    1387722600  # 2013-12-22 14:30
]

In [10]:
val_indices = []
index = 0
for index, row in train2.iterrows():
    time = row['TIMESTAMP']
    latitude = row['LATITUDE']
    for ts in cuts:
        if time <= ts and time + 15 * (len(latitude) - 1) >= ts:
            val_indices.append(index)
            break
    index += 1

saving and reloading validation indexes

In [23]:
with open(os.path.join(path, "val_idx.pkl"), "wb") as f:
    pickle.dump(val_indices, f)

In [11]:
with open(os.path.join(path, "val_idx.pkl"), "rb") as f:
    val_indices = pickle.load(f)

validation set

For making embedding layers, I will be needing indexes to feed as values. It will be used for doing lookup.

Before making validation set, this is something I am doing after getting stuck with some error in `fit`. I noticed that for `WEEK_OF_YEAR` and for `ORIGIN_STAND`, the cat codes do no match with index. For these 2, I will replace values with indexes. This is an experiment which is unlikely to change anything. Just trying it out -- **NOPE DIDN'T HELP**. Commented below 3 chunks

In [16]:
for c in cat_vars:
    s =0
    for i, v in enumerate(train2[c].unique()):
        s += i != v
    print(s, c)

In [96]:
# def EmbeddingDataPreprocess(data, cats, inplace=True):
#     ### Each categorical column should have indices as values
#     ### Which will be looked up at embedding matrix and used in modeling
#     ### Make changes inplace
#     if inplace:
#         for c in cats:
#             data[c].replace({val: i for i, val in enumerate(data[c].unique())}, inplace=True)
#         return data
#     else:
#         data_copy = data.copy()
#         for c in cats:
#             data_copy[c].replace({val: i for i, val in enumerate(data_copy[c].unique())}, inplace=True)
#         return data_copy

In [97]:
# EmbeddingDataPreprocess(train2, ['ORIGIN_STAND', 'WEEK_OF_YEAR'], inplace=True)

In [98]:
# for c in cat_vars:
#     s =0
#     for i, v in enumerate(train2[c].unique()):
#         s += i != v
#     print(s, c)

**Index == values** now

Found that in below codes, index of embedding for `WEEK_OF_YEAR` starts from 1 and others from 0. I am going to change that to make it start from 0 because I guess it is the reason of `cude runtime error` that I am getting. Experiment!!


In [12]:
# making week of year from 0-51 from 1-52

train2.WEEK_OF_YEAR = train2.WEEK_OF_YEAR - 1

In [13]:
# transformation on test set
test2.WEEK_OF_YEAR = test2.WEEK_OF_YEAR - 1

In [14]:
X_valid = train2.iloc[val_indices]

Let's check if we have right timestamps in validation set or not

In [15]:
lt = []
for d in X_valid['TIMESTAMP']:
    lt.append(datetime.date.fromtimestamp(d))
    
set(lt)

{datetime.date(2013, 8, 14),
 datetime.date(2013, 10, 1),
 datetime.date(2013, 10, 7),
 datetime.date(2013, 11, 2),
 datetime.date(2013, 12, 22)}

Yup. above dates match with our dates of test set. Just 1 previous year

In [16]:
X_train = train2.drop(train2.index[[val_indices]])

In [17]:
len(X_valid), len(X_train)

(304, 1710366)

Validation set length is similar as test set length

In [18]:
with open(os.path.join(path, "arrival-clusters.pkl"), "rb") as f:
    cluster_centers2 = pickle.load(f)

In [19]:
len(cluster_centers2)

3422

Saving `X_train` and `X_valid`

In [20]:
X_train.to_pickle(path + 'X_train.pkl')
X_valid.to_pickle(path + 'X_valid.pkl')

Reading validation and training files. Next time can directly read from here

In [184]:
X_train = pd.read_pickle('X_train.pkl')
X_valid = pd.read_pickle('X_valid.pkl')

### Step 4: DL

Hidden layer is basically telling probability of observation being in ith cluster. So it will have as many outputs as number of clusters. (3422). It includes training for `probabilities`  

Now sum of all the `probabilities` should be 1. Therefore we will use `softmax` over a trained linear layer


On these 3422 output from hidden, we take weighted avg which will just be sum of multiplied probability into cluster center (i.e. sum of 3422 elements). It includes no traning, just using previous probabilities, multiplying by fixed crntroid coordinated and calculating weighted avg.   

Think of this operation as a linear layer with `weights = centers (i.e. fixed) and input = probabilities`



Layers to add:  
1. Pass input data to a **dense layer** which has 500 outputs and those **500 to relu**. 
1. Pass coming data to a **linear layer** which has 3422 outputs
2. These 3422 outputs passed to a **softmax** (which will give p)
3. These 3422 p's will go to **linear layer** which **does not** update weights.  just multiply p and c . 
4. 


#### Analysing cardinality for embeddings

In [20]:
cols = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR']

In [21]:
for i in cols:
    print(i, '--', 'Possible values', len(train2[i].unique()) ,'||', 'Embedding size', 10)

ORIGIN_CALL -- Possible values 57106 || Embedding size 10
TAXI_ID -- Possible values 448 || Embedding size 10
ORIGIN_STAND -- Possible values 64 || Embedding size 10
HOUR -- Possible values 24 || Embedding size 10
DAY_OF_WEEK -- Possible values 7 || Embedding size 10
WEEK_OF_YEAR -- Possible values 52 || Embedding size 10


They took embedding of size 10 for all the above features

In [22]:
emb_szs = []
for i in cols:
    emb_szs.append((len(train2[i].unique()), 10))
emb_szs

[(57106, 10), (448, 10), (64, 10), (24, 10), (7, 10), (52, 10)]

In [23]:
cat_vars = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR']
#cont_vars = 

#### Getting continuous features

In [25]:
X_train[:1]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,TARGET,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,1372636858620000589,2,0,0,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[-0.21451, -0.214974, -0.199688, -0.182087, -0...","[-0.0437321, -0.0412145, -0.0731591, -0.105104...","[-8.63084, 41.1545]","[-0.0437321, -0.0412145, -0.0731591, -0.105104...",0,0,26


In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
# replacing NA with 0s
# k = 5

X_train.COORD_FEATURES.fillna(0, inplace = True)
X_valid.COORD_FEATURES.fillna(0, inplace = True)

In [26]:
test2.COORD_FEATURES.fillna(0, inplace = True)

In [27]:
def get_features(df, target = True):
    
    cols = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR','MISSING_DATA']
    
    df_feat = df[cols]
    for i in range(10):
        df_feat[f'long_{i}'] = df.COORD_FEATURES.apply(lambda x: 0 if type(x) == int else x[i]) # creating 10 cols for long
        df_feat[f'lat_{i}'] = df.COORD_FEATURES.apply(lambda x: 0 if type(x) == int else x[i+10]) # 10 cols for lat

    if target is True:
        df_target = df['TARGET']
    else:
        df_target = None
               
    return df_feat, df_target

Training and validation sets

In [28]:
train2.COORD_FEATURES.fillna(0, inplace = True)
df, y = get_features(train2)

In [29]:
train_feat, train_target = get_features(X_train)

In [30]:
valid_feat, valid_target = get_features(X_valid) 

In [31]:
test_feat, test_target = get_features(test2, target = False)

Centers of cluster lats and longs. Needed for last nn layer

In [32]:
cluster_lats = []

for i in cluster_centers2:
    cluster_lats.append(i[0])
    
cluster_longs = []

for i in cluster_centers2:
    cluster_longs.append(i[1])

In [33]:
cluster_lats = np.array(cluster_lats)[:,None]
cluster_longs = np.array(cluster_longs)[:,None]

Mixed input nn model

#### Data loader

Customizing `fast.ai` functions

#### After loading data into dataloader, writing my own training loop (tried `get_learner` from fastai. doesn't work for me)

In [34]:
class taxi_dataset_from_df():
    def __init__(self, df, cats, y):
        #n = len(cats[0]) if cats else len(conts[0])
        
        self.df_cats = df[cats]
        self.df_conts = df.drop(cats, axis=1)
        
        lat = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[1])
        long = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[0])
        
        target = pd.DataFrame({'long': long.astype(np.float32),'lat' : lat.astype(np.float32)})
        
        cat_cols = [c.values for n,c in self.df_cats.items()]
        cont_cols = [c.values for n,c in self.df_conts.items()]  
        target_cols = [c.values for n,c in target.items()]  
        
        self.cats = np.stack(cat_cols, 1).astype(np.int64)
        self.conts = np.stack(cont_cols, 1).astype(np.float32) 
        self.target = np.stack(target_cols, 1).astype(np.float32) 

    def __len__(self): return len(self.df_cats)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.target[idx]]


In [35]:
train_dataset = taxi_dataset_from_df(train_feat, cat_vars, train_target)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)

In [36]:
valid_feat.shape

(304, 27)

In [37]:
valid_dataset = taxi_dataset_from_df(valid_feat, cat_vars, valid_target)

valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=304, shuffle=False)

In [38]:
class taxi_dataset_from_df_test():
    def __init__(self, df, cats):
        #n = len(cats[0]) if cats else len(conts[0])
        
        self.df_cats = df[cats]
        self.df_conts = df.drop(cats, axis=1)
        
        #lat = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[1])
        #long = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[0])
        
        #target = pd.DataFrame({'long': long.astype(np.float32),'lat' : lat.astype(np.float32)})
        
        cat_cols = [c.values for n,c in self.df_cats.items()]
        cont_cols = [c.values for n,c in self.df_conts.items()]  
        #target_cols = [c.values for n,c in target.items()]  
        
        self.cats = np.stack(cat_cols, 1).astype(np.int64)
        self.conts = np.stack(cont_cols, 1).astype(np.float32) 
       # self.target = np.stack(target_cols, 1).astype(np.float32) 

    def __len__(self): return len(self.df_cats)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx]]


In [39]:
test2.shape

(320, 15)

In [40]:
test_dataset = taxi_dataset_from_df_test(test_feat, cat_vars)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=320, shuffle=False)

let's see what my dataset is returning. It is a list with 3 elements. cat df, cont df and y.

**NOTICE that lat is in 0 here and long is in 1**

In [41]:
train_dataset[0:3]

[array([[ 0,  0,  0,  0,  0, 26],
        [ 0,  1,  7,  0,  0, 26],
        [ 0,  2,  0,  0,  0, 26]]),
 array([[ 0.     , -0.04373, -0.21451, -0.04121, -0.21497, -0.07316, -0.19969, -0.1051 , -0.18209, -0.13656,
         -0.17452, -0.26228, -0.03808, -0.25677, -0.03783, -0.25677, -0.03772, -0.25677, -0.03793, -0.25694,
         -0.03808],
        [ 0.     , -0.41443,  0.03392, -0.42325,  0.03453, -0.45549,  0.03783, -0.49499,  0.04292, -0.53811,
          0.0491 , -0.93476,  0.14369, -0.90834,  0.15754, -0.88284,  0.17128, -0.86757,  0.17977, -0.86711,
          0.18023],
        [ 0.     ,  0.05555, -0.22872,  0.04832, -0.22882,  0.03368, -0.2298 ,  0.02393, -0.22856,  0.0041 ,
         -0.22748, -0.13893, -0.17334, -0.09062, -0.18765, -0.05774, -0.2071 , -0.02139, -0.22192,  0.003  ,
         -0.2264 ]], dtype=float32),
 array([[ 41.15449,  -8.63084],
        [ 41.17067,  -8.66574],
        [ 41.14053,  -8.61597]], dtype=float32)]

In [42]:
train_dataset[:][0]

array([[  0,   0,   0,   0,   0,  26],
       [  0,   1,   7,   0,   0,  26],
       [  0,   2,   0,   0,   0,  26],
       ..., 
       [  0, 439,   0,  10,   4,   0],
       [  0, 443,  12,  15,   0,  26],
       [  0, 444,  34,  19,   0,  26]])

In [113]:
test_dataset[0:3]

[array([[  0, 150,  15,  17,   3,  32],
        [  0, 306,  57,  17,   3,  32],
        [  0, 260,  15,  17,   3,  32]]),
 array([[ 0.     ,  0.5326 , -0.11858,  0.53197, -0.11698,  0.53245, -0.1141 ,  0.53167, -0.11312,  0.52725,
         -0.11261,  0.52569, -0.1159 ,  0.52128, -0.12769,  0.50948, -0.13289,  0.51687, -0.137  ,  0.54646,
         -0.14421],
        [ 0.     ,  0.09205, -0.15841,  0.09237, -0.15826,  0.09158, -0.15574,  0.0996 , -0.15002,  0.11707,
         -0.14236,  0.20991,  0.05347,  0.21383,  0.06392,  0.23098,  0.07447,  0.23758,  0.07828,  0.24907,
          0.08482],
        [ 0.     ,  0.5315 , -0.11806,  0.53167, -0.11446,  0.53182, -0.1125 ,  0.5219 , -0.11189,  0.5249 ,
         -0.11796,  0.67532,  0.10937,  0.67831,  0.11369,  0.68522,  0.12321,  0.70348,  0.13386,  0.72095,
          0.14045]], dtype=float32)]

In [43]:
np.min(test_dataset[:][0], 0), np.max(test_dataset[:][0], 0)

(array([ 0,  0,  0,  2,  0, 32]),
 array([47093,   444,    63,    17,     6,    50]))

In [44]:
np.min(train_dataset[:][0], 0), np.max(train_dataset[:][0], 0)

(array([0, 0, 0, 0, 0, 0]), array([57105,   447,    63,    23,     6,    51]))

In [45]:
np.min(valid_dataset[:][0], 0), np.max(valid_dataset[:][0], 0)

(array([ 0,  0,  0,  3,  0, 32]),
 array([34587,   435,    63,    17,     6,    50]))

Ok. So let's think on how will I use input of these for in dataloader and that into my `Net`

#### Defining Loss function

They used `equirectangular distance` as loss function which performed better than original kaggle metric (`Haversine distance`). Actually equirectangular distance is just a simpler approximation to haversine distance

Will keep long in 0 and lat in 1 as there in target column

In [194]:
# a predicted, b true
# data loader is in such a way that true y should be indexed from floatTensor

class equiRectLoss(nn.Module):

    def __init__(self):
        super(equiRectLoss, self).__init__()
        self.rearth = 6371
        self.deg2rad = 3.141592653589793 / 180.

    def forward(self, a,b):  
        lat1 = a[:,0]
        lon1 = a[:,1]
        lat2 = b[:,0]
        lon2 = b[:,1]   
        
        x = (lon2-lon1) * torch.cos((lat1+lat2)/2)
        y = (lat2-lat1)

        loss = torch.mean((torch.sqrt(x**2 + y**2) * self.rearth),dim=0)
        return loss

#### defining nn

In [54]:
cluster_latsV = torch.Tensor(cluster_lats)
cluster_longsV = torch.Tensor(cluster_longs)
clus_latlong = torch.cat((cluster_latsV, cluster_longsV), dim=1)

In [55]:
clus_latlong.shape

torch.Size([3422, 2])

In [49]:
def emb_init(x):
    x = x.weight.data
    sc = 2/(x.size(1)+1)
    x.uniform_(-sc,sc)

In [50]:
def final_layer_wt(x):
    x = x.weight.data
    x = clus_latlong
    return x

In [114]:
# they didn't do batchnormm in paper            
class Net(nn.Module):
    def __init__(self, emb_szs, n_cont,
                 y_range=None, use_bn=False, clus_latlong= V(clus_latlong)):
        super().__init__()
        
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        #n_emb = sum(e.embedding_dim for e in self.embs)
        n_emb = 60
        
        # 6 cat vars . therefore 60 embeddings (10 each)
        self.lin1 = nn.Linear(n_emb + n_cont, 500) # 1st linear layer -- no. features x 500
        self.relu = nn.ReLU()   # 2nd relu layer (output = 500 neurons)
        self.lin2 = nn.Linear(500, 3422)  # 3rd linear layer. 500x3422
        self.soft = nn.Softmax() # 4th softmax layer. 3422 probabilities (will sum to 1)
         
        for o in [self.lin1, self.lin2] : kaiming_normal(o.weight.data) #weight initialization for lin layers
            
        self.bn = nn.BatchNorm1d(n_cont) 
        
       # self.lin3 = nn.Linear(3422,2)
        
        #for o in [self.lin3]: final_layer_wt(o)
        #for o in [self.lin3] : kaiming_normal(o.weight.data)

    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
        x = torch.cat(x, 1)
        x2 = self.bn(x_cont)
        x = torch.cat([x, x2], 1)
        
        out = self.lin1(x)
        out = self.relu(out)
        out = self.lin2(out)
        out = self.soft(out)
#         outlat = out.mm(cluster_latsV)  # out is n*3422 and cluster_lats is 3422*1
#         outlong = out.mm(cluster_longsV)
        #out = self.lin3(out)
        
        out = V(torch.mm(out,V(clus_latlong)))
        out.retain_grad()

        #target = pd.DataFrame({'long': outlong.astype(np.float32),'lat' : outlat.astype(np.float32)})
        #target_cols = [c.values for n,c in target.items()]  
        #target_out = torch.cat((cluster_latsV, cluster_longsV), dim=1)

        return out # nx2 tensor where n is no. of observations

In [115]:
it = iter(train_loader)
a,b,c = next(it)

In [116]:
a.shape, b.shape, c.shape

(torch.Size([200, 6]), torch.Size([200, 21]), torch.Size([200, 2]))

In [117]:
def score(x, y):
    y_pred = to_np(net2(V(x)))
    return np.sum(y_pred.argmax(axis=1) == to_np(y))/len(y_pred)

In [128]:
net1 = Net(emb_szs= emb_szs, n_cont= len(df.columns)-len(cat_vars), y_range=y)
net1 = net1.cuda()
loss= F.mse_loss
learning_rate = 1e-3 # can try different values, cyclic and diff learning rates
#optimizer=optim.SGD(net1.parameters(), lr=learning_rate, momentum = 0.9)
optimizer=optim.Adam(net1.parameters(), lr=learning_rate) # will check if weight_decay needed or not later


for epoch in range(3):
    losses=[]
    losses_val = []
    running_loss = 0.0
    #running_loss_valid = 0.0
    examples = 0
    #dl = iter(train_loader)
    for i, data in enumerate(train_loader, 0):
    #for t in range(len(dl)):
        # Forward pass: compute predicted y and loss by passing x to the model.
        xcat, xcont, yt = data      
        y_pred = net1(V(xcat), V(xcont))
        l = loss(y_pred, V(yt))
        
#         xcat_val, xcont_val, yt_val = valid_dataset[:]
#         l_valid = loss(net1(V(T(xcat_val)), V(T(xcont_val))), V(T(yt_val)))
        
#         losses_val.append(l_valid)
        
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable weights of the model)
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model parameters
        l.backward()

        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()
        
        # Print statistics
        running_loss += l.data[0]
        #running_loss_valid += l_valid.data[0]
        examples += 10
        
        print('[%d, %5d] Training loss: %.3f' % (epoch + 1, i + 1, running_loss / examples))
        
        #print('[%d, %5d] Training loss: %.3f Validation loss: %.3f' % (epoch + 1, i + 1, running_loss / examples, 
                                                 # running_loss_valid/examples))
        
print('Finished Training')

[1,     1] Training loss: 3.096
[1,     2] Training loss: 2.654
[1,     3] Training loss: 2.654
[1,     4] Training loss: 2.543
[1,     5] Training loss: 2.211
[1,     6] Training loss: 1.990
[1,     7] Training loss: 1.958
[1,     8] Training loss: 2.045
[1,     9] Training loss: 2.162
[1,    10] Training loss: 2.343
[1,    11] Training loss: 2.291
[1,    12] Training loss: 2.211
[1,    13] Training loss: 2.075
[1,    14] Training loss: 2.021
[1,    15] Training loss: 2.122
[1,    16] Training loss: 2.017
[1,    17] Training loss: 1.950
[1,    18] Training loss: 1.989
[1,    19] Training loss: 1.978
[1,    20] Training loss: 1.901
[1,    21] Training loss: 1.894
[1,    22] Training loss: 1.909
[1,    23] Training loss: 1.883
[1,    24] Training loss: 1.842
[1,    25] Training loss: 1.892
[1,    26] Training loss: 1.870
[1,    27] Training loss: 1.882
[1,    28] Training loss: 1.894
[1,    29] Training loss: 1.889
[1,    30] Training loss: 1.856
[1,    31] Training loss: 1.867
[1,    3

[1,   261] Training loss: 1.815
[1,   262] Training loss: 1.811
[1,   263] Training loss: 1.810
[1,   264] Training loss: 1.810
[1,   265] Training loss: 1.812
[1,   266] Training loss: 1.805
[1,   267] Training loss: 1.804
[1,   268] Training loss: 1.806
[1,   269] Training loss: 1.804
[1,   270] Training loss: 1.806
[1,   271] Training loss: 1.803
[1,   272] Training loss: 1.800
[1,   273] Training loss: 1.807
[1,   274] Training loss: 1.805
[1,   275] Training loss: 1.804
[1,   276] Training loss: 1.803
[1,   277] Training loss: 1.801
[1,   278] Training loss: 1.800
[1,   279] Training loss: 1.804
[1,   280] Training loss: 1.804
[1,   281] Training loss: 1.799
[1,   282] Training loss: 1.808
[1,   283] Training loss: 1.808
[1,   284] Training loss: 1.805
[1,   285] Training loss: 1.803
[1,   286] Training loss: 1.803
[1,   287] Training loss: 1.801
[1,   288] Training loss: 1.798
[1,   289] Training loss: 1.798
[1,   290] Training loss: 1.796
[1,   291] Training loss: 1.795
[1,   29

[1,   535] Training loss: 1.747
[1,   536] Training loss: 1.746
[1,   537] Training loss: 1.745
[1,   538] Training loss: 1.742
[1,   539] Training loss: 1.743
[1,   540] Training loss: 1.741
[1,   541] Training loss: 1.745
[1,   542] Training loss: 1.745
[1,   543] Training loss: 1.745
[1,   544] Training loss: 1.743
[1,   545] Training loss: 1.744
[1,   546] Training loss: 1.743
[1,   547] Training loss: 1.745
[1,   548] Training loss: 1.747
[1,   549] Training loss: 1.747
[1,   550] Training loss: 1.747
[1,   551] Training loss: 1.748
[1,   552] Training loss: 1.747
[1,   553] Training loss: 1.748
[1,   554] Training loss: 1.749
[1,   555] Training loss: 1.747
[1,   556] Training loss: 1.746
[1,   557] Training loss: 1.745
[1,   558] Training loss: 1.744
[1,   559] Training loss: 1.745
[1,   560] Training loss: 1.745
[1,   561] Training loss: 1.743
[1,   562] Training loss: 1.744
[1,   563] Training loss: 1.743
[1,   564] Training loss: 1.744
[1,   565] Training loss: 1.743
[1,   56

[1,   810] Training loss: 1.739
[1,   811] Training loss: 1.740
[1,   812] Training loss: 1.739
[1,   813] Training loss: 1.739
[1,   814] Training loss: 1.739
[1,   815] Training loss: 1.739
[1,   816] Training loss: 1.741
[1,   817] Training loss: 1.742
[1,   818] Training loss: 1.741
[1,   819] Training loss: 1.742
[1,   820] Training loss: 1.740
[1,   821] Training loss: 1.740
[1,   822] Training loss: 1.741
[1,   823] Training loss: 1.741
[1,   824] Training loss: 1.741
[1,   825] Training loss: 1.740
[1,   826] Training loss: 1.741
[1,   827] Training loss: 1.741
[1,   828] Training loss: 1.741
[1,   829] Training loss: 1.741
[1,   830] Training loss: 1.741
[1,   831] Training loss: 1.743
[1,   832] Training loss: 1.744
[1,   833] Training loss: 1.743
[1,   834] Training loss: 1.743
[1,   835] Training loss: 1.743
[1,   836] Training loss: 1.743
[1,   837] Training loss: 1.743
[1,   838] Training loss: 1.745
[1,   839] Training loss: 1.744
[1,   840] Training loss: 1.746
[1,   84

[1,  1085] Training loss: 1.747
[1,  1086] Training loss: 1.746
[1,  1087] Training loss: 1.747
[1,  1088] Training loss: 1.747
[1,  1089] Training loss: 1.746
[1,  1090] Training loss: 1.745
[1,  1091] Training loss: 1.745
[1,  1092] Training loss: 1.745
[1,  1093] Training loss: 1.745
[1,  1094] Training loss: 1.744
[1,  1095] Training loss: 1.743
[1,  1096] Training loss: 1.744
[1,  1097] Training loss: 1.744
[1,  1098] Training loss: 1.744
[1,  1099] Training loss: 1.743
[1,  1100] Training loss: 1.744
[1,  1101] Training loss: 1.743
[1,  1102] Training loss: 1.742
[1,  1103] Training loss: 1.742
[1,  1104] Training loss: 1.743
[1,  1105] Training loss: 1.741
[1,  1106] Training loss: 1.742
[1,  1107] Training loss: 1.742
[1,  1108] Training loss: 1.742
[1,  1109] Training loss: 1.741
[1,  1110] Training loss: 1.741
[1,  1111] Training loss: 1.741
[1,  1112] Training loss: 1.741
[1,  1113] Training loss: 1.740
[1,  1114] Training loss: 1.740
[1,  1115] Training loss: 1.740
[1,  111

...

[3,  7323] Training loss: 1.693
[3,  7324] Training loss: 1.693
[3,  7325] Training loss: 1.693
[3,  7326] Training loss: 1.693
[3,  7327] Training loss: 1.693
[3,  7328] Training loss: 1.693
[3,  7329] Training loss: 1.693
[3,  7330] Training loss: 1.693
[3,  7331] Training loss: 1.693
[3,  7332] Training loss: 1.693
[3,  7333] Training loss: 1.693
[3,  7334] Training loss: 1.693
[3,  7335] Training loss: 1.693
[3,  7336] Training loss: 1.693
[3,  7337] Training loss: 1.693
[3,  7338] Training loss: 1.693
[3,  7339] Training loss: 1.693
[3,  7340] Training loss: 1.693
[3,  7341] Training loss: 1.693
[3,  7342] Training loss: 1.693
[3,  7343] Training loss: 1.693
[3,  7344] Training loss: 1.693
[3,  7345] Training loss: 1.693
[3,  7346] Training loss: 1.693
[3,  7347] Training loss: 1.693
[3,  7348] Training loss: 1.693
[3,  7349] Training loss: 1.693
[3,  7350] Training loss: 1.693
[3,  7351] Training loss: 1.693
[3,  7352] Training loss: 1.693
[3,  7353] Training loss: 1.693
[3,  735

[3,  7598] Training loss: 1.693
[3,  7599] Training loss: 1.693
[3,  7600] Training loss: 1.693
[3,  7601] Training loss: 1.694
[3,  7602] Training loss: 1.694
[3,  7603] Training loss: 1.694
[3,  7604] Training loss: 1.694
[3,  7605] Training loss: 1.694
[3,  7606] Training loss: 1.694
[3,  7607] Training loss: 1.694
[3,  7608] Training loss: 1.694
[3,  7609] Training loss: 1.694
[3,  7610] Training loss: 1.694
[3,  7611] Training loss: 1.694
[3,  7612] Training loss: 1.694
[3,  7613] Training loss: 1.694
[3,  7614] Training loss: 1.694
[3,  7615] Training loss: 1.694
[3,  7616] Training loss: 1.694
[3,  7617] Training loss: 1.694
[3,  7618] Training loss: 1.694
[3,  7619] Training loss: 1.694
[3,  7620] Training loss: 1.694
[3,  7621] Training loss: 1.694
[3,  7622] Training loss: 1.694
[3,  7623] Training loss: 1.694
[3,  7624] Training loss: 1.694
[3,  7625] Training loss: 1.694
[3,  7626] Training loss: 1.694
[3,  7627] Training loss: 1.694
[3,  7628] Training loss: 1.694
[3,  762

[3,  7873] Training loss: 1.695
[3,  7874] Training loss: 1.695
[3,  7875] Training loss: 1.695
[3,  7876] Training loss: 1.695
[3,  7877] Training loss: 1.695
[3,  7878] Training loss: 1.695
[3,  7879] Training loss: 1.695
[3,  7880] Training loss: 1.695
[3,  7881] Training loss: 1.695
[3,  7882] Training loss: 1.695
[3,  7883] Training loss: 1.695
[3,  7884] Training loss: 1.695
[3,  7885] Training loss: 1.695
[3,  7886] Training loss: 1.695
[3,  7887] Training loss: 1.695
[3,  7888] Training loss: 1.695
[3,  7889] Training loss: 1.695
[3,  7890] Training loss: 1.695
[3,  7891] Training loss: 1.695
[3,  7892] Training loss: 1.695
[3,  7893] Training loss: 1.695
[3,  7894] Training loss: 1.695
[3,  7895] Training loss: 1.695
[3,  7896] Training loss: 1.694
[3,  7897] Training loss: 1.695
[3,  7898] Training loss: 1.694
[3,  7899] Training loss: 1.694
[3,  7900] Training loss: 1.694
[3,  7901] Training loss: 1.694
[3,  7902] Training loss: 1.694
[3,  7903] Training loss: 1.694
[3,  790

[3,  8148] Training loss: 1.693
[3,  8149] Training loss: 1.693
[3,  8150] Training loss: 1.694
[3,  8151] Training loss: 1.694
[3,  8152] Training loss: 1.693
[3,  8153] Training loss: 1.694
[3,  8154] Training loss: 1.694
[3,  8155] Training loss: 1.694
[3,  8156] Training loss: 1.694
[3,  8157] Training loss: 1.694
[3,  8158] Training loss: 1.694
[3,  8159] Training loss: 1.694
[3,  8160] Training loss: 1.694
[3,  8161] Training loss: 1.694
[3,  8162] Training loss: 1.694
[3,  8163] Training loss: 1.694
[3,  8164] Training loss: 1.694
[3,  8165] Training loss: 1.693
[3,  8166] Training loss: 1.693
[3,  8167] Training loss: 1.693
[3,  8168] Training loss: 1.693
[3,  8169] Training loss: 1.693
[3,  8170] Training loss: 1.693
[3,  8171] Training loss: 1.693
[3,  8172] Training loss: 1.693
[3,  8173] Training loss: 1.693
[3,  8174] Training loss: 1.693
[3,  8175] Training loss: 1.693
[3,  8176] Training loss: 1.693
[3,  8177] Training loss: 1.693
[3,  8178] Training loss: 1.693
[3,  817

[3,  8423] Training loss: 1.694
[3,  8424] Training loss: 1.694
[3,  8425] Training loss: 1.694
[3,  8426] Training loss: 1.694
[3,  8427] Training loss: 1.694
[3,  8428] Training loss: 1.694
[3,  8429] Training loss: 1.694
[3,  8430] Training loss: 1.694
[3,  8431] Training loss: 1.694
[3,  8432] Training loss: 1.694
[3,  8433] Training loss: 1.694
[3,  8434] Training loss: 1.694
[3,  8435] Training loss: 1.693
[3,  8436] Training loss: 1.694
[3,  8437] Training loss: 1.694
[3,  8438] Training loss: 1.694
[3,  8439] Training loss: 1.693
[3,  8440] Training loss: 1.693
[3,  8441] Training loss: 1.693
[3,  8442] Training loss: 1.693
[3,  8443] Training loss: 1.693
[3,  8444] Training loss: 1.694
[3,  8445] Training loss: 1.693
[3,  8446] Training loss: 1.694
[3,  8447] Training loss: 1.694
[3,  8448] Training loss: 1.694
[3,  8449] Training loss: 1.694
[3,  8450] Training loss: 1.694
[3,  8451] Training loss: 1.694
[3,  8452] Training loss: 1.694
[3,  8453] Training loss: 1.694
[3,  845

In [ ]:
# def print_stats(epoch, names, values):
#     print_str = "{}: {:3d}, ".format("epoch",epoch)
#     print_str += ', '.join(("{}: {:.6f}".format(n, v) for (n, v) in zip(names, values)))
#     print(print_str)  

In [123]:
net1.parameters

<bound method Module.parameters of Net (
  (embs): ModuleList (
    (0): Embedding(57106, 10)
    (1): Embedding(448, 10)
    (2): Embedding(64, 10)
    (3): Embedding(24, 10)
    (4): Embedding(7, 10)
    (5): Embedding(52, 10)
  )
  (lin1): Linear (81 -> 500)
  (relu): ReLU ()
  (lin2): Linear (500 -> 3422)
  (soft): Softmax ()
  (bn): BatchNorm1d(21, eps=1e-05, momentum=0.1, affine=True)
)>

In [124]:
# for i, data in enumerate(test_loader, 0):
#     xcat_test, xcont_test = data      
#     test_pred = net1(V(xcat_test), V(xcont_test))

In [125]:
xcat_test, xcont_test = test_dataset[:]
test_pred = net1(V(T(xcat_test)), V(T(xcont_test)))

In [126]:
test_pred

Variable containing:
 41.1616  -8.6712
 41.1163  -8.6593
 41.1616  -8.6712
 41.1149  -8.6589
 41.1609  -8.6710
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1149  -8.6589
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1149  -8.6589
 41.1587  -8.6704
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1149  -8.6589
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1149  -8.6589
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1578  -8.6702
 41.1595  -8.6707
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1616  -8.6712
 41.1613  -8.6711
 41.1616  -8.6712
 41.1616  -8.6712
 41.161